<a href="https://colab.research.google.com/github/kwakseoyeon/1test/blob/master/building_a_llm_math_chain_based_on_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
# 문제
# LLMChain을 이용해서 LLMMathChain과 유사하게 만들어보기!

'''
1. 체인 만들기 (prompt_engineering)
chain.invoke("What is 13 raised to the .3432 power?")

output :

```text
13**0.3432
```

2. 13**0.3432 파싱해서, 실행 numexpr.evaluate( xxx ) 결과를 얻음
'''

'''
LLM 프롬프트 예상

사용자가 입력한 문장을 numexpr.evaluate에 적합하도록 작성합니다.
예시
입력 : "What is 13 raised to the .3432 power?"
출력 :
```text
13**0.3432
```

출력양식
```text
numexpr.evaluate에 넣을 인자값
```
'''

'\nLLM 프롬프트 예상\n\n사용자가 입력한 문장을 numexpr.evaluate에 적합하도록 작성합니다.\n예시\n입력 : "What is 13 raised to the .3432 power?"\n출력 :\n```text\n13**0.3432\n```\n\n출력양식\n```text\nnumexpr.evaluate에 넣을 인자값\n```\n'

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

chat_model = ChatOpenAI(temperature=0)

template= """You are a helpful assistant that make an only ARGUMENT for numexpr.evaluate according to user query such as
```text
13**0.3432
```
when user query is 'What is 13 raised to the .3432 power?'

Please response as code block using text back quote like
```text
13**0.3432
```
without other description."""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chain = LLMChain(llm = chat_model,
                 prompt = chat_prompt_template)

chain.run(text = "What is 15 raised to the .3432 power?")

'```text\n15**0.3432\n```'

In [ ]:
from langchain.schema import BaseOutputParser

# ```text\n15**0.3432\n``` > 15**0.3432

class CodeOutputParser(BaseOutputParser):
    def parse(self, text: str):
        value = text.strip().split("```")
        print(value)
        return value[1].split("\n")[1]

output = chain.run(text="What is 15 raised to the .3432 power?")
output = CodeOutputParser().parse(output)

print(output)

['', 'text\n15**0.3432\n', '']
15**0.3432


In [ ]:
import numexpr

In [ ]:
numexpr.evaluate('15**0.3432')

array(2.53299609)

In [ ]:
numexpr.evaluate(output)

array(2.53299609)

In [ ]:
def math_llm(input_text):
    output = chain.run(text=input_text)
    output = CodeOutputParser().parse(output)
    return str(numexpr.evaluate(output))

math_llm("What is 15 raised to the .3432 power?")

['', 'text\n15**0.3432\n', '']


'2.5329960941138205'

In [ ]:
'''
"You are a helpful assistant that make an argument for numexpr.
evaluate according to user query such as\n```text\n13**0.3432\n```
 when user query is 'What is 13 raised to the .3432 power?'
 Please response as code block using text back quote like ```text\n13**0.3432\n```
 without other description."
'''

my_math_chain = LLMChain(llm = chat_model,
                         prompt = chat_prompt_template,
                         output_parser = CodeOutputParser(),
                         verbose = True)

output = my_math_chain.run(text="What is 15 raised to the .3432 power?")

print(numexpr.evaluate(output))



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant 
that make an argument for numexpr.evaluate according to user query such as
```text
13**0.3432
``` 
when user query is 'What is 13 raised to the .3432 power?' 

Please response as code block using text back quote like 
```text
13**0.3432
``` 
without other description.
Human: What is 15 raised to the .3432 power?
['', 'text\n15**0.3432\n', '']

> Finished chain.
2.5329960941138205
